<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#A-tool-to-visualize-trajectory-frames:-ViewManager" data-toc-modified-id="A-tool-to-visualize-trajectory-frames:-ViewManager-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>A tool to visualize trajectory frames: ViewManager</a></span><ul class="toc-item"><li><span><a href="#Example---show-all-neighbours-of-a-structure" data-toc-modified-id="Example---show-all-neighbours-of-a-structure-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Example - show all neighbours of a structure</a></span></li><li><span><a href="#Example---show-PT-structures-with-5-largest-and-5-smallest-potential-energies" data-toc-modified-id="Example---show-PT-structures-with-5-largest-and-5-smallest-potential-energies-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Example - show PT structures with 5 largest and 5 smallest potential energies</a></span></li><li><span><a href="#Example---plot-an-eigenvector-of-SqRA" data-toc-modified-id="Example---plot-an-eigenvector-of-SqRA-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Example - plot an eigenvector of SqRA</a></span></li></ul></li><li><span><a href="#Classify-full-trajectories-in-cells-of-FullGrid" data-toc-modified-id="Classify-full-trajectories-in-cells-of-FullGrid-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Classify full trajectories in cells of FullGrid</a></span></li><li><span><a href="#Assigning-trajectories-to-a-FullGrid-cell" data-toc-modified-id="Assigning-trajectories-to-a-FullGrid-cell-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Assigning trajectories to a FullGrid cell</a></span><ul class="toc-item"><li><span><a href="#Test:-assigning-PT-to-a-FullGrid" data-toc-modified-id="Test:-assigning-PT-to-a-FullGrid-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Test: assigning PT to a FullGrid</a></span></li><li><span><a href="#Assigning-a-real-trajectory-fo-a-FullGrid" data-toc-modified-id="Assigning-a-real-trajectory-fo-a-FullGrid-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Assigning a real trajectory fo a FullGrid</a></span></li></ul></li></ul></div>

In [1]:
import sys
import os

try:
    os.chdir(r"/home/hanaz63/PAPER_MOLECULAR_ROTATIONS_2022/nobackup/molgri")
    sys.path.append(r"/home/hanaz63/PAPER_MOLECULAR_ROTATIONS_2022/nobackup")
except FileNotFoundError:
    os.chdir(r"D:\HANA\phD\PAPER_2022\molecularRotationalGrids")
    sys.path.append(r"D:\HANA\phD\PAPER_2022\molecularRotationalGrids")
    
import warnings
warnings.filterwarnings("ignore")

In [2]:
import nglview as nv
import MDAnalysis as mda
import numpy as np
import time
from scipy.sparse import csr_array
from numpy.typing import NDArray
import matplotlib
import pandas as pd
import matplotlib.pyplot as plt
from IPython.core.display import display

from molgri.molecules.transitions import SimulationHistogram, MSM, SQRA
from molgri.plotting.molecule_plots import TrajectoryPlot
from molgri.molecules.parsers import FileParser, ParsedEnergy, XVGParser
from molgri.paths import PATH_OUTPUT_PT, PATH_OUTPUT_LOGGING, PATH_OUTPUT_ENERGIES
from molgri.space.fullgrid import FullGrid
from molgri.space.utils import k_argmin_in_array, k_argmax_in_array
from molgri.plotting.widgets import ViewManager, sync_all_views, display_all_views

## A tool to visualize trajectory frames: ViewManager

- in Jupyter, using nglview
- input: mda.Universe and one or more indices along the trajectory
- output: frames plotted sequentially or overlapping, color/opacity indicating a property, moving simutanously

In [3]:
fg_assigning = FullGrid("8", "12", "linspace(0.2, 1, 5)")
sh_pt = SimulationHistogram("H2O_H2O_0179", is_pt=True, second_molecule_selection="bynum 4:6",
                           use_saved=False)

sh_traj = SimulationHistogram("H2O_H2O_0095_2000", is_pt=False, second_molecule_selection="bynum 4:6",
                             full_grid=fg_assigning, use_saved=False)

TypeError: SimulationHistogram.__init__() missing 1 required positional argument: 'reference_name'

### Example - show all neighbours of a structure

In [4]:
# EXAMPLE - neighbours
chosen_cell = 17
print(f"Our chosen cell is {chosen_cell} with position index {sh_pt.full_grid.get_position_index(chosen_cell)}",
f"and quaternion_index {sh_pt.full_grid.get_quaternion_index(chosen_cell)}")


# display the structure assigned the chosen_cell
vm = ViewManager(sh_pt.trajectory_universe)
view1 = vm.plot_frames_overlapping(np.where(sh_pt.get_full_assignments==chosen_cell))
display(view1)


# display its neighbours - limiting here to 20 structures to not overwhelm the program
# if they are neighbours in position grid, they should have the same quaternion index
vm.fresh_view()
neighbouring_frames = sh_pt.get_indices_neighbours_of_cell_i(chosen_cell)
print("Neighbours")
print("Pindex", sh_pt.full_grid.get_position_index(neighbouring_frames))
print("Qindex", sh_pt.full_grid.get_quaternion_index(neighbouring_frames))
print(f"There are {len(neighbouring_frames)} neighbours of cell {chosen_cell}: {neighbouring_frames}")

vm.plot_frames_sequential(neighbouring_frames)

Our chosen cell is 17 with position index 0 and quaternion_index 17


NGLWidget()

Neighbours
Pindex [ 0  0  0  0  0  0  0  0  0  0 20 26 26 30 42]
Qindex [31 28 34  2  1 33  3 27 21  7 22  3 27  7 22]
There are 15 neighbours of cell 17: [  31   28   34    2    1   33    3   27   21    7  822 1043 1067 1207
 1702]


Box(children=(NGLWidget(layout=Layout(width='200px')), NGLWidget(layout=Layout(width='200px')), NGLWidget(layo…

### Example - show PT structures with 5 largest and 5 smallest potential energies

In [12]:
# EXAMPLE - 


# display
vm = ViewManager(sh_pt.trajectory_universe)
vm.fresh_view()

# magnitudes
magnitudes = sh_pt.get_magnitude_energy("Potential")


# 5 structures with largest energies
num_extremes = 20
argmax_index = k_argmax_in_array(magnitudes, num_extremes)
vm.set_color_magnitude(magnitudes)
view1 = vm.plot_frames_overlapping(argmax_index, opacities=0.8)

# 5 structures with smallest energies
vm.fresh_view()
argmin_index = k_argmin_in_array(magnitudes, num_extremes)
vm.set_color_magnitude(magnitudes)
view2 = vm.plot_frames_overlapping(argmin_index, opacities=0.8)

# show both
sync_all_views([view1, view2])
display_all_views([view1, view2])

Box(children=(NGLWidget(), NGLWidget()), layout=Layout(display='inline-flex', flex_flow='row wrap', width='100…

### Example - plot an eigenvector of SqRA

In [13]:
# EXAMPLE - eigenvector 0

# display
vm = ViewManager(sh_pt.trajectory_universe)
vm.fresh_view()

# magnitudes
magnitudes = sh_pt.get_magnitude_ith_eigenvector(0)
vm.set_opacity_magnitude(magnitudes)



# largest absolute values
num_extremes = 20
argmax_index = k_argmax_in_array(np.abs(magnitudes), num_extremes)

vm.plot_frames_overlapping(argmax_index)
vm.view

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "C:\Users\Hana\AppData\Local\Programs\Python\Python39\lib\site-packages\IPython\core\interactiveshell.py", line 3508, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\Hana\AppData\Local\Temp\ipykernel_15616\3854418558.py", line 8, in <module>
    magnitudes = sh_pt.get_magnitude_ith_eigenvector(0)
  File "D:\HANA\phD\PAPER_2022\molecularRotationalGrids\molgri\molecules\transitions.py", line 165, in get_magnitude_ith_eigenvector
    my_eigenvector = evec[0].T[i]
  File "D:\HANA\phD\PAPER_2022\molecularRotationalGrids\molgri\molecules\transitions.py", line 187, in get_transition_model
    else:
  File "D:\HANA\phD\PAPER_2022\molecularRotationalGrids\molgri\molecules\transitions.py", line 441, in __init__
  File "D:\HANA\phD\PAPER_2022\molecularRotationalGrids\molgri\molecules\transitions.py", line 286, in __init__
    self.num_cells = len(self.sim_hist.full_grid.get_full_grid_as_array())
  File "D:\HANA\phD\PAPER

## Classify full trajectories in cells of FullGrid

In [10]:
# find assignments to particular position class
selected_class = 4
in_this_position_class = sh_traj.get_indices_same_position(selected_class)
print(f"There are {len(in_this_position_class )} frames in this position class.")
# you can use the indices in VMD to visualise all
print(in_this_position_class)

vm = ViewManager(sh_traj.trajectory_universe)
vm.fresh_view()

vm.plot_frames_overlapping(np.random.choice(in_this_position_class, 30))
vm.view

There are 26 frames in this position class.
[1494 1495 1496 1497 1498 1499 1500 1501 1751 1752 1753 1754 1755 1756
 1757 1758 1759 1760 1761 1762 1763 1764 1765 1766 1767 1768]


NGLWidget()

In [11]:
# find assignments to particular orientation class

selected_class = 3
in_this_orientation_class = sh_traj.get_indices_same_orientation(selected_class)
print(f"There are {len(in_this_orientation_class )} frames in this orientation class.")
print(in_this_orientation_class)

vm.fresh_view()
vm.plot_frames_overlapping(np.random.choice(in_this_orientation_class, 30))
vm.view


There are 227 frames in this orientation class.
[ 437  438  439  440  441  442  443  463  464  465  466  467  468  469
  470  471  472  473  474  475  476  477  478  479  480  481  482  483
  484  485  486  487  488  489  490  491  492  493  494  495  553  554
  555  556  557  558  559  560  561  562  563  564  565  566  567  568
  569  570  571  572  573  574  575  752  769  770  771  772  773  774
  775  776  777  778  779  780  781  782  783  784  785  786  787  788
  789  790  791  792  793 1050 1051 1052 1053 1054 1055 1056 1057 1058
 1059 1060 1061 1062 1063 1064 1065 1066 1067 1068 1069 1070 1071 1072
 1073 1074 1080 1081 1082 1083 1084 1085 1086 1087 1088 1089 1090 1091
 1092 1093 1094 1095 1096 1097 1098 1099 1100 1101 1102 1103 1104 1105
 1190 1191 1192 1193 1194 1195 1591 1592 1593 1594 1595 1596 1597 1598
 1599 1600 1601 1602 1603 1604 1605 1606 1607 1608 1609 1610 1611 1612
 1613 1614 1615 1616 1617 1618 1619 1620 1621 1622 1623 1624 1625 1626
 1627 1733 1734 1735 1736 173

NGLWidget()

In [12]:
# find assignments to one cell of full grid
selected_class = 4*8+3
in_this_class = sh_pt.get_indices_same_cell(selected_class)
print(f"There are {len(in_this_class )} frames in this cell.")
print(in_this_class)

vm.fresh_view()
vm.plot_frames_overlapping(in_this_class, opacities=0.8)
vm.view

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "C:\Users\Hana\AppData\Local\Programs\Python\Python39\lib\site-packages\IPython\core\interactiveshell.py", line 3508, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\Hana\AppData\Local\Temp\ipykernel_15616\1528756530.py", line 3, in <module>
    in_this_class = sh_pt.get_indices_same_cell(selected_class)
  File "D:\HANA\phD\PAPER_2022\molecularRotationalGrids\molgri\molecules\transitions.py", line 151, in get_indices_same_cell
  File "D:\HANA\phD\PAPER_2022\molecularRotationalGrids\molgri\molecules\transitions.py", line 181, in get_full_assignments
    return self.full_assignments
  File "D:\HANA\phD\PAPER_2022\molecularRotationalGrids\molgri\molecules\transitions.py", line 270, in _assign_trajectory_2_full_grid
  File "D:\HANA\phD\PAPER_2022\molecularRotationalGrids\molgri\molecules\transitions.py", line 176, in get_quaternion_assignments
    return self.quaternion_assignments
  File "D:\HANA\phD\PAPER_2022\m

In [ ]:

# TODO: plot eigenvectors and see if they are sensible
# TODO: network flow plot for strongest rates?

# TODO: make transition matrix determination faster and try it for a large matrix
# TODO: HF forcefield and calculations

## Assigning trajectories to a FullGrid cell
- input: (pseudo)trajectory and a FullGrid of choice
- assign structures to position, orientation or full classes (cells of FullGrid) 


### Test: assigning PT to a FullGrid

- if assigning to its own fg, assignments should be 1:1
- if assigning to a smaller fg, an uniform number per cell is expected

In [ ]:
evaluation_fg = FullGrid("40", "42", "linspace(0.2, 1.5, 10)")

# this pt was generated with fg = FullGrid("40", "42", "linspace(0.2, 1.5, 10)")
my_pt_name = "H2O_H2O_0179"

sh = SimulationHistogram(my_pt_name, full_grid=evaluation_fg, second_molecule_selection="bynum 4:6")
np.set_printoptions(threshold=sys.maxsize)

In [ ]:
assignments = sh.get_position_assignments()
natural_numbers = np.array(range(len(sh.full_grid.get_position_grid_as_array())))
natural_numbers = np.repeat(natural_numbers, sh.full_grid.b_rotations.get_N())

assert np.all(assignments == natural_numbers)

assignments = sh.get_quaternion_assignments()
natural_numbers = np.array(range(sh.full_grid.b_rotations.get_N()))
natural_numbers = np.tile(natural_numbers, len(sh.full_grid.get_position_grid_as_array()))

for el1, el2 in zip(assignments, natural_numbers):
    print(el1, el2)
    #pass

#assignments = sh.get_full_assignments()
#natural_numbers = np.array(range(len(assignments)))

#for el1, el2 in zip(assignments, natural_numbers):
#    print(el1, el2)

### Assigning a real trajectory fo a FullGrid

In [ ]:
my_fg = FullGrid("8", "12", "[0.3, 0.6, 0.9, 1.2]")
sh_traj = SimulationHistogram("H2O_H2O_0095_2000", full_grid=my_fg, second_molecule_selection="bynum 4:6")


In [ ]:
# most populated cells 
assignments = sh_traj.get_full_assignments()
my_indices, my_counts = np.unique(assignments.astype(int), return_counts=True)
most_populated_cells = my_indices[k_argmax_in_array(my_counts, 5)]
vm = ViewManager(sh.trajectory_universe)
for selected_class in most_populated_cells:
    vm.fresh_view()
    vm.plot_frames_overlapping(np.where(assignments==selected_class)[0][::10])
    display(vm.view)

In [ ]:
# are the most populated cells also the ones with lowest average energy?
# maybe not because the size of the cell strongly affects the population
all_popular_frames = []
full_assignments = sh_traj.get_full_assignments().astype(int)
my_indices, my_counts = np.unique(full_assignments, return_counts=True)
most_populated_cells = my_indices[k_argmax_in_array(my_counts, 3)]
for populated_cell in most_populated_cells:
    belongs_to_this = np.where(sh_traj.get_full_assignments()==populated_cell)[0]
    all_popular_frames.extend(belongs_to_this)

cell_volumes = np.array(sh_traj.full_grid.get_total_volumes())
volume_per_assignment = cell_volumes[full_assignments]

pot_energy = energies.get_energies("Potential")/volume_per_assignment
print("Popular frames energy", np.min(pot_energy[all_popular_frames]), np.average(pot_energy[all_popular_frames]),
     np.max(pot_energy[all_popular_frames]))
print("All frames energy", np.min(pot_energy), np.average(pot_energy),
     np.max(pot_energy))

In [ ]:
from molgri.plotting.transition_plots import TransitionPlot
my_msm = MSM(sh, energies)
my_tp = TransitionPlot(my_msm)
my_tp.plot_heatmap(save=False)

In [ ]:
my_tp.plot_its(save=False)